In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install konlpy

학습 데이터

In [13]:
import pandas as pd 

path = '/content/drive/MyDrive/'
chatbotData = pd.read_csv(path+'ChatData.csv')
question, answer = list(chatbotData["Q"]) , list(chatbotData["A"])
print(len(question))
print(len(answer))
print(" ")

for i in range(10):
    print("질문:" + question[i])
    print("답변:" + answer[i])
    print(" ")
    

11823
11823
 
질문:12시 땡!
답변:하루가 또 가네요.
 
질문:1지망 학교 떨어졌어
답변:위로해 드립니다.
 
질문:3박4일 놀러가고 싶다
답변:여행은 언제나 좋죠.
 
질문:3박4일 정도 놀러가고 싶다
답변:여행은 언제나 좋죠.
 
질문:PPL 심하네
답변:눈살이 찌푸려지죠.
 
질문:SD카드 망가졌어
답변:다시 새로 사는 게 마음 편해요.
 
질문:SD카드 안돼
답변:다시 새로 사는 게 마음 편해요.
 
질문:SNS 맞팔 왜 안하지ㅠㅠ
답변:잘 모르고 있을 수도 있어요.
 
질문:SNS 시간낭비인 거 아는데 매일 하는 중
답변:시간을 정하고 해보세요.
 
질문:SNS 시간낭비인데 자꾸 보게됨
답변:시간을 정하고 해보세요.
 


태깅

In [14]:
import re
import matplotlib.pyplot as plt
import numpy as np
import os
from konlpy.tag import Okt 
from keras import models, layers, optimizers, metrics, preprocessing

#태그 단어
PAD = '<PADDING>' # 패딩 
STA = '<START>'   # 시작 
END = '<END>'     # 끝
OOV = '<OOV>'     # out of vocabulary 
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

ENCODER_INPUT = 0
DECODER_INPUT = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
maxSequences = 30

# 임베딩 벡터 차원
embeddingDim = 100

# LSTM 출력 차원
lstmHiddenDim = 128

# 정규표현식 필터
RE_FILTER = re.compile("[.,!?\':;~()]'")
# 먼저 파이썬에서 정규표현식을 사용하기 위해서는 Regex를 위한 모듈인 re 모듈을 사용한다.
# re 모듈의 compile 함수는 정규식 패턴을 입력으로 받아들여 정규식 객체를 리턴하는데, 
# 즉 re.compile(검색할문자열) 와 같이 함수를 호출하면 정규식 객체 (re.RegexObject 클래스 객체)를 리턴하게 된다.

형태소 분석기

In [15]:
# 형태소 분석기 
def posTag(sentences): 
    tagger = Okt()
    sentencePos = []
    for sentence in sentences:
        # 특수문자 제거 
        sentence = re.sub( RE_FILTER  ,  ""  , sentence )
        sentence = " ".join(tagger.morphs(sentence))
        sentencePos.append(sentence)
    return sentencePos 
    
question = posTag(question)
answer = posTag(answer)

In [16]:
# 질문 + 대답 을 하나로 합치기 
sentences = [] 
sentences.extend(question)
sentences.extend(answer)
len(sentences)

23646

단어 배열

In [17]:
words = []
# 단어배열 생성 
for sentence in sentences : 
    for word in sentence.split():
        words.append(word)


# words에서 길이가 0인 단어를 삭제 
# 중복단어를 삭제 
words = [ word for word in words if len(word)>0 ] # 길이가 0인 단어 삭제 
words = list(set(words)) # 중복 단어 삭제 

words[:0] = [PAD,STA,END,OOV]

# 단어에 대한 인덱스를 부여 -> 딕셔너리
wordToIndex = {word:index for index, word in enumerate(words)}
indexToWord = {index:word for index, word in enumerate(words)}
indexToWord

{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '다니시나',
 5: '말랐는지',
 6: '푸르른',
 7: '되요',
 8: '주무셨나',
 9: '코스',
 10: '채우면',
 11: '무엇',
 12: '오셨길',
 13: '주년',
 14: '만남이네요',
 15: '괜',
 16: '따라와주지',
 17: '벌이',
 18: '이브',
 19: '설레겠어요',
 20: '때리기',
 21: '챙겨',
 22: '8일',
 23: '벌면',
 24: '텔레비전',
 25: '대보',
 26: '있어줄게요',
 27: '흘렀는데',
 28: '찾아오려나',
 29: '같네',
 30: '필요해요',
 31: '털어도',
 32: '월동',
 33: '잠드는데',
 34: '오길',
 35: '얄',
 36: '가보다',
 37: '지금',
 38: '무서운건',
 39: '나아질',
 40: '돌아오고',
 41: '달라지는게',
 42: '많았겠네요',
 43: '기한',
 44: '쓰임',
 45: '체조',
 46: '식는',
 47: '멍청한거지',
 48: '오히려',
 49: '없애는',
 50: '반의',
 51: '저주',
 52: '힘드네',
 53: '걸핏하면',
 54: '아니라네',
 55: '있음',
 56: '막말',
 57: '숨길',
 58: '보러가는',
 59: '마시고',
 60: '설치',
 61: '셀프',
 62: '사귀는데',
 63: '약해졌나',
 64: '달라지겠죠',
 65: '남길',
 66: '상하',
 67: '다가왔습니다',
 68: '호호',
 69: '받지도',
 70: '잡았대',
 71: '지나갈',
 72: '헛소리',
 73: '였겠어요',
 74: '사실이네요',
 75: '아니란',
 76: '애프터',
 77: '씩',
 78: '일치',
 79: '3일',
 80: '냉장고',
 81: '가슴',
 82: '비세요',

문장 인코딩
(인덱스로 변환)

In [18]:
# 문장 -> 인덱스로 변환
def convertTextToIndex(sentences, voc, mytype):
    
    sentencesIndex=[]
    for sentence in sentences:
        
        sentenceIndex=[]
        if mytype == DECODER_INPUT:
            
            sentenceIndex.extend([voc[STA]])
            
        for word in sentence.split():
            
            if voc.get(word) is not None: # 단어에 해당하는 인덱스가 있는 경우
                sentenceIndex.extend([voc[word]]) # 단어에 해당되는 인덱스가 추가
                
            else: # 사전에 없는 단어의 경우 OOV추가
                sentenceIndex.extend([voc[OOV]])
                
        if mytype == DECODER_TARGET:
            
            # 디코더 출력은 맨 마지막에 end 추가
            if maxSequences <= len(sentenceIndex):
                sentenceIndex = sentenceIndex[:maxSequences-1] + [voc[END]]
                
            else:
                sentenceIndex += [voc[END]]
                
        else:
            if len(sentenceIndex) > maxSequences:
                sentenceIndex = sentenceIndex[:maxSequences]
                
        # 0으로 채움(pad_sequence)
        sentenceIndex += [wordToIndex[PAD]] * (maxSequences-len(sentenceIndex))
        
        sentencesIndex.append(sentenceIndex)
            
    return np.asarray(sentencesIndex)

In [19]:
# 인코더 입력, 디코더 입력, 디코더 출력 -> 인덱스 변환
xEncoder = convertTextToIndex(question, wordToIndex, ENCODER_INPUT)
print(xEncoder[0])

xDecoder = convertTextToIndex(answer, wordToIndex, DECODER_INPUT)
print(xDecoder[0])

yDecoder = convertTextToIndex(answer, wordToIndex, DECODER_TARGET)
print(yDecoder[0])

[8239 2168  460    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[   1 3284 6621 6482 1725 2430    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[3284 6621 6482 1725 2430    2    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


#원핫 인코딩

In [ ]:
# np.zeros((2,3,4))
oneHotData=np.zeros((len(yDecoder),maxSequences,len(words)))
# 100(답변개수), 30(최대 단어 개수), 454(전체 단어 집합 개수)

np.shape(oneHotData)#디코더 출력
# 하루 가 또 가네요 END
for i, seq in enumerate(yDecoder):# (100,30)
    for j, index in enumerate(seq):
        oneHotData[i,j,index]=1
yDecoder=oneHotData

모델 생성

In [ ]:
# 훈련 모델 생성
# 인코더 정의

# 입력 문장의 인덱스 sequence를 입력
encoderInputs = layers.Input(shape=(None))
# 임베딩 계층 
encoderOutputs = layers.Embedding(len(words),embeddingDim)(encoderInputs)

encoderOutputs,stateH, stateC=layers.LSTM(lstmHiddenDim,return_state=True, 
            dropout=0.2, recurrent_dropout=0.5)(encoderOutputs)
#return_state=True => 상태값 리턴
#LSTM은 2개 상태 존재(셀, 히든 스테이트)

encoderStates=[stateH, stateC]


#디코더 정의
#출력 문장의 인덱스 sequence를 입력
decoderInputs=layers.Input(shape=(None,))
#임베딩 계층
decoderEmbedding=layers.Embedding(len(words),
                                embeddingDim)
decoderOutputs=decoderEmbedding(decoderInputs)



decoderLSTM=layers.LSTM(lstmHiddenDim,
                        return_state=True, 
            return_sequences=True, 
                        dropout=0.2, 
                        recurrent_dropout=0.5)
decoderOutputs, _, _=decoderLSTM(decoderOutputs,initial_state=encoderStates)
decoderDense=layers.Dense(len(words), 
                          activation="softmax")
decoderOutputs=decoderDense(decoderOutputs)

In [ ]:
model=models.Model([encoderInputs, decoderInputs],
             decoderOutputs)
             
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
             
#예측 모델 인코더 정의
encoderModel=models.Model(encoderInputs, 
                          encoderStates)

#예측 모델 디코더 정의
#바로 앞에 있는 디코더의 출력(상태)을 입력 받아서
#예측을 해야 함.
decoderStateInputH=layers.Input(shape=(lstmHiddenDim,))
decoderStateInputC=layers.Input(shape=(lstmHiddenDim,))
decoderStatesInputs=[decoderStateInputH,decoderStateInputC]

#임베딩 계층
decoderOutputs=decoderEmbedding(decoderInputs)
#LSTM 계층
decoderOutputs, stateH, stateC=decoderLSTM(decoderOutputs,
           initial_state=decoderStatesInputs)
decoderStates=[stateH, stateC]

#Dense계층을 통해 원핫 형식으로 예측 단어 인덱스를 추출
decoderOutputs=decoderDense(decoderOutputs)

#예측 모델 디코더 설정
decoderModel=models.Model([decoderInputs]+decoderStatesInputs,
            [decoderOutputs]+decoderStates)

인덱스를 문장으로 변환

In [ ]:
#인덱스를 문장으로 변환
def convertIndexToText(indexs, voc):
    #구현
    sentence=""
    for i in indexs:
        if i==END_INDEX: #종료 인덱스
            break;
        if voc.get(i) is not None:
            sentence+=voc[i]
        else:
            sentence.extend([voc[OOV_INDEX]])
        sentence +=" "    
    return sentence    

#에폭
for epoch in range(10):
    print("total epoch:", epoch+1)
    history=model.fit([xEncoder,xDecoder], yDecoder,
             epochs=100,
             batch_size=64,
                     verbose=0)
    print("accuracy :", history.history['accuracy'])
    print("loss :", history.history['loss'])
    #문장 예측
    #3박 4일 놀러 가고 싶다 -> 여행 은 언제나 좋죠
    
    inputEncoder=xEncoder[2].reshape(1,xEncoder[2].shape[0]) #(30,) ->(1,30)
    inputDecoder=xDecoder[2].reshape(1,xDecoder[2].shape[0]) #(30,) ->(1,30)
    
    results=model.predict([inputEncoder,inputDecoder])
    
    #결과값에 대해서 가장 큰 값의 위치를 구함
    index=np.argmax(results[0], 1)
    #인덱스 -> 문장으로 변환
    sentence=convertIndexToText(index, indexToWord)
    print(sentence)
    print()

In [ ]:
# 문장 입력
#[[320, 157, ..., 19,0,0,0,0...,0]]
def makePredictInput(sentence):    
    sentences=[]
    sentences.append(sentence)
    sentences=posTag(sentences)
    inputSeq=convertTextToIndex(sentences,
                                wordToIndex,
                                ENCODER_INPUT)
    return inputSeq
    
makePredictInput("3박4일 놀러가고 싶다")

>>> 
array([[ 4552,  4334, 12220,   893,  1422,   608,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0]])

예측 답변 생성

In [ ]:
#예측 답변 생성
def generateText(inputSeq):
        #입력을 인코더에 넣고, 마지막 상태 구함
        states=encoderModel.predict(inputSeq)
        
        #목표 시퀀스 초기화
        targetSeq=np.zeros((1,1))
        #<START> 시그널을 추가
        targetSeq[0,0]=STA_INDEX
        #인덱스 초기화
        indexs=[]
        
        #디코더 반복
        while 1:
            decoderOuputs, stateH, stateC=decoderModel.predict([targetSeq]+states)
            #결과를 원핫인코딩 형식으로 변환
            index=np.argmax(decoderOuputs[0,0,:])
            indexs.append(index)
            #종료 체크
            if index==END_INDEX or len(indexs)>=maxSequences:
                break
                
                #targetSeq를 이전 출력으로 설정
            targetSeq=np.zeros((1,1))
            targetSeq[0,0]=index    #STA_INDEX 
            
            #디코더의 이전 상태를 다음 디코더 예측에 사용
            states=[stateH, stateC]
            
        #인덱스를 문장으로 변환
        sentence=convertIndexToText(indexs, indexToWord)
        
        return sentence 

In [ ]:
#입력문장 -> 인덱스
inputSeq=makePredictInput("놀러가고 싶다")
#[[320, 157, ..., 19,0,0,0,0...,0]]

sentence=generateText(inputSeq)
sentence #여행은 언제나 좋죠

>>> '단 짠으로 두 개 사는게 진리 죠 . '